# Fetching Tweets in Real Time using Twitter API

Tweepy library is used to route all the Live tweets to a TCP socket server.<br> 
This server is hosted in same local machine.

**Import tweepy library**

In [1]:
! pip3 install tweepy --user

In [1]:
import tweepy

**Import OAuthHandler**<br> Used for Authentication protocol (OAuth) of Twitter API.

In [2]:
from tweepy import OAuthHandler

**Import Stream & StreamListner**<br> Listen to streaming data getting from the socket.

In [3]:
from tweepy import Stream
from tweepy.streaming import StreamListener

**Import socket** <br> It is used to create socket connection with Localhost i.e. local server. <br>
**Import Json** <br>Json is used because twitter data comes in json format.

In [4]:
import socket
import json

To connect to Twitter API, we need these 4 Keys. <br>
It can be generated from https://apps.twitter.com/ by creating a new app.

In [5]:
consumer_key = 'UM51djtI4OXAZ6VlrCDjTQWs5'
consumer_secret = 'E4fyt0c8EqXUiOi0ELjO06Hywzb3BITegs0U8NWbih34gzhy8g'
access_token = '785846632726294530-NL8MoBRW9Ku48F7w47QOWsCugg1mz9u'
access_secret = 'hzS6uUNmR41ymX11BNLLcRRF1q4nfo6QzPOnYPgjsFsOU'

#### Class TweetsListner
    1.listening to the stream via StreamListener. 
    2.constrctor function (init) : initializing the socket. 
    3.tweet_data contains the tweet's json coming from the stream. 
    4.In the tweet's json object 'text' field contains the actual Tweet.
    5.The actual tweet is extracted & sent to the client socket.
    6.on_error is used to catch & throw errors.

In [6]:
class TweetsListener(StreamListener):
  def __init__(self, csocket):
      self.client_socket = csocket
  def on_data(self, tweet_json):
      try:
          tweet_data = json.loads( tweet_json )
          print("tweet_text: ", tweet_data['text'].encode('utf-8') )
          print("created_at: ", tweet_data['created_at'])  
          print("name: ", tweet_data['user']['name'])              
          print("location: ", tweet_data['user']['location'])
          print("\n")
          s = tweet_data['text'] + "\n"
          print(s)
          s = s.encode('utf-8')
          self.client_socket.send( s )
          return True
      except BaseException as e:
          print("Error on_data: %s" % str(e))
      return True
  def on_error(self, status):
      print(status)
      return True

### Function send_twitter_data():
    1. auth is doing Authentication check using OAuthHandler. It uses 4 keys of Twitter APIs.
    2. consumer_key & consumer_secret are like username & access_token & access_secret are like password.
    3. twitter_stream will get the actual twitter live stream data.
    4. It'll call Stream with auth & call the Class TweetsListner & pass c_socket info to the class.
    5. From the stream of tweets, filter & take only tweets which contains "corona" word.

In [7]:
def send_twitter_data(c_socket):
  auth = OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_secret)
  twitter_stream_data = Stream(auth, TweetsListener(c_socket))
  twitter_stream_data.filter(track=['corona'])

### Socket Creation:
    1. Socket is created by using socket function.
    2. Host is localhost, which is machine IP, where this Jupyter notebook is running.
    3. Port is 7773 (It can be anything, unique for this service)
    4. Now bind host & port address to the socket.

In [8]:
# Create a socket object
s = socket.socket()

# Get local machine name
host = "localhost"

# Reserve a port for your service.
port = 7777

# Bind to the port
s.bind((host, port))

# print
print("Listening on port: %s" % str(port))
print("Listening on port: %s" % str(host))

Listening on port: 7777
Listening on port: localhost


socket will wait & listen for few seconds. <br>
After that connection is made.

In [9]:
# Now wait for client connection.
s.listen(5)

In [10]:
# Establish connection with client.
c, addr = s.accept()

In [11]:
print("Recieved request ",  str(addr))

Recieved request  ('127.0.0.1', 59314)


### Send live Tweets in real time to the Socket
Function send_twitter_data will send all the Tweets containing word 'corona' to socket 'c' in real time.<br>
This will be consumed in the next code of Spark Streaming to do Trend Analysis.

In [12]:
send_twitter_data(c)

tweet_text:  b'RT @miob1781: Aufpassen: Wer Impfp\xc3\xa4sse f\xc3\xa4lscht oder gef\xc3\xa4lschte Impfp\xc3\xa4sse benutzt, macht sich strafbar und kann ins Gef\xc3\xa4ngnis gehen. #Impfzw\xe2\x80\xa6'
created_at:  Wed May 05 10:11:15 +0000 2021
name:  Susanne Jonda
location:  Germany, Bülstedt


RT @miob1781: Aufpassen: Wer Impfpässe fälscht oder gefälschte Impfpässe benutzt, macht sich strafbar und kann ins Gefängnis gehen. #Impfzw…

tweet_text:  b'RT @DrPuerner: Was war das nur f\xc3\xbcr ein Aufschrei als ich sagte, dass das Gesundheitssystem zu keiner Zeit \xc3\xbcberlastet war. #Corona \n\xf0\x9f\x91\x87\n\xe2\x80\x9eDie M\xe2\x80\xa6'
created_at:  Wed May 05 10:11:15 +0000 2021
name:  Kerstin Meier
location:  None


RT @DrPuerner: Was war das nur für ein Aufschrei als ich sagte, dass das Gesundheitssystem zu keiner Zeit überlastet war. #Corona 
👇
„Die M…

tweet_text:  b'RT @TrimZeeShah: Dial \xf0\x9f\x93\x9e 9123\nStay at Home\n&amp; Get yourself vaccinated\n\n#SindhGovt has ta

ProtocolError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))